In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def h(x, alpha):
    return 1-x[0]**2-alpha*x[1]**2

def u_of_x(x, u_min, u_max, k=2.6, alpha=1):
    assert x.shape[0]==2, 'shape_error'
    assert u_min < u_max, f'input constraint error. {u_min} > {u_max}?'
    al = alpha
    m = 1
    l = 1
    g = 10.
    gamma = pow(10, k)
    th, thd = x
    if thd == 0:
        p = abs(m*g*np.sin(th)/2)
        ran = [u_min, -p] if th > 0 else [p, u_max]
    else:
        u_thres = np.roots([-(6*al*thd)/(m*l**2), -(2*th*thd + 3*al*g/l*thd*np.sin(th) - gamma*h(x, al))])[0]
        if thd < 0:
            if u_thres > u_max:
                print('no cbf compensation')
            u_thres = max(u_thres, u_min)
            ran = [u_thres, u_max]
        else:
            if u_thres < u_min:
                print('no cbf compensation')
            u_thres = min(u_thres, u_max)
            ran = [u_min, u_thres]
    return ran

In [56]:
def u_cbf(u_candidate, ran):
    rmin, rmax = ran[0], ran[1]
    out = u_candidate
    if u_candidate < rmin:
        out = rmin
    elif u_candidate > rmax:
        out = rmax
    else:
        pass
    return out

def rad_clip(th):
    r = abs(th)
    while r > np.pi:
        r -= np.pi
    th = np.sign(th) * r
    return th

def dynamics(x, u, dt=0.05):
    m = 1
    l = 1
    g = 10.
    assert x.shape[0] == 2, 'aho'
    th, thdot = x
    th = th + thdot*dt + (3*g)/(2*l)*np.sin(th)*dt**2 + 3/(m*l**2)*u*dt**2
    thdot = thdot + (3*g)/(2*l)*np.sin(th)*dt + 3/(m*l**2)*u*dt
    th, thdot = rad_clip(th), rad_clip(thdot)
    return np.array([th, thdot])

In [31]:
# check cbf condition
import csv

region = []

with open('./CBF/safe_region.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        region.append(row)
region = np.array(region).astype('float64')

In [51]:
h_values = []

for i, p in enumerate(region):
    h_values.append([h(p, 0.4)])
h_values = np.array(h_values)
region_and_h = np.hstack((region, h_values))

region_and_h = region_and_h[np.argsort(region_and_h[:, 2])]

edge = region_and_h[np.random.permutation(np.arange(300))[0:200], 0:2]

edge = edge[np.argsort(edge[:, 0])]

for p in edge:
    print(p, u_of_x(p, -10., 10., alpha=0.4))

[-0.99006819 -0.16353134] [-4.19273056505361, 10.0]
[-0.98863393 -0.08674161] [-10.0, 10.0]
[-0.98849297 -0.21295885] [1.306824242779866, 10.0]
[-0.98781883 -0.28018073] [9.252032149747219, 10.0]
[-0.98506601 -0.22219848] [-2.4003042006428967, 10.0]
[-0.98473291  0.0879831 ] [-10.0, 10.0]
[-0.97811923  0.28778237] [-10.0, 10.0]
[-0.97765995  0.16941641] [-10.0, 10.0]
[-0.97089524 -0.30538635] [-5.959005113237587, 10.0]
[-0.96509622  0.54089764] [-10.0, -9.940049309748705]
[-0.96238256  0.52753276] [-10.0, -6.88568016492866]
[-0.95604181 -0.46043727] [4.455063586013983, 10.0]
[-0.95043951  0.60040792] [-10.0, -8.27129503164805]
[-0.94724871 -0.45399856] [-2.559104043270809, 10.0]
[-0.94575077  0.51609834] [-10.0, 4.525413850132039]
[-0.93710636  0.52798628] [-10.0, 8.053621378853785]
[-0.93600212  0.56530479] [-10.0, 3.6534423345952414]
[-0.93561012 -0.58952269] [8.850986311704288, 10.0]
[-0.93353577  0.68444194] [-10.0, -9.471663415718954]
[-0.930965    0.51454721] [-10.0, 10.0]
[-0.91

In [60]:
input_rule = np.clip(np.random.randn(10000,) * 10, -10., 10.)

In [67]:
x = np.array([-0.99006819, -0.16353134])
for i in range(10000):
    u_candidate = input_rule[i]
    u = u_cbf(u_candidate, u_of_x(x, -10., 10., alpha=0.4))
    print(u, u_candidate, u_of_x(x, -10., 10., alpha=0.4))
    x = dynamics(x, u, dt=0.0005)
    print(x, h(x, 0.4))
    assert h(x,0.4) >= 0, f'danger state : {x}, h(x) = {h(x, 0.4)}'

-4.192729709679552 -7.071827811624609 [-4.192729709679552, 10.0]
[-0.99015624 -0.17609127] 0.0071873749226185266
5.336733858578004 5.336733858578004 [-1.7648188927508786, 10.0]
[-0.99024341 -0.17435737] 0.007257784424378948
8.63747531284352 8.63747531284352 [-1.8988220881901596, 10.0]
[-0.99032725 -0.1676727 ] 0.008006284439955475
1.1908576030176956 1.1908576030176956 [-2.9142878751781733, 10.0]
[-0.99041333 -0.1721583 ] 0.0072260447462536605
5.997768032265301 5.997768032265301 [-1.9558314578257763, 10.0]
[-0.99049805 -0.1694339 ] 0.007430482995598748
0.7620306917255923 0.7620306917255923 [-2.2676286940649057, 10.0]
[-0.99058533 -0.17456345] 0.006551749445028742
-1.2185421130379077 -10.0 [-1.2185421130379077, 10.0]
[-0.99067666 -0.18266424] 0.005213266170056456
0.27336541628952543 -7.01263091625128 [0.27336541628952543, 10.0]
[-0.99077092 -0.18852756] 0.00415592167500641
3.6908842932364214 3.6908842932364214 [1.3512540167251474, 10.0]
[-0.99086556 -0.18926499] 0.0038569569408312323
1.6

2.9309960145965 -2.137049448857162 [2.9309960145965, 10.0]
[-0.99984322 -0.01894107] 0.00017002312126888886
3.9401535721191565 3.9401535721191565 [3.551142862579752, 10.0]
[-0.99985289 -0.01934128] 0.00014455648934568954
8.759574921075565 8.759574921075565 [3.800398222466502, 10.0]
[-0.99985915 -0.01251238] 0.00021905613238427696
10.0 10.0 [2.1361392692300427, 10.0]
[-0.99986106 -0.00382285] 0.00027201201616516076
-2.781263724537899 -2.781263724537899 [-6.76272849436651, 10.0]
[-0.99986821 -0.01430524] 0.0001816984159585559
2.933318831828156 -1.5076772815351778 [2.933318831828156, 10.0]
[-0.99987632 -0.0162158 ] 0.0001421598997610302
10.0 10.0 [3.5860394853157307, 10.0]
[-0.99988009 -0.00752634] 0.00021715697382461126
3.1023719231899345 3.1023719231899345 [0.25419970173625794, 10.0]
[-0.99988468 -0.00918335] 0.00019689949121867115
1.4837034481296536 -10.0 [1.4837034481296536, 10.0]
[-0.99989131 -0.01326839] 0.00014694615118015166
8.251347752306346 8.251347752306346 [3.203220674073843, 

AssertionError: danger state : [-0.99999297 -0.01074877], h(x) = -3.214776121985576e-05

In [5]:
for p in region:
    u_c = np.clip(np.random.randn() * 10, -10., 10.)
    x = np.zeros((2,))
    x[0], x[1] = float(p[0]), float(p[1])
    u = u_cbf(u_c, u_of_x(x, -10., 10., alpha=0.4))
    print(f'x = {x}, u = {u}, x^ = {dynamics(x,u)}')
    assert h(dynamics(x,u), 0.4) >= 0, f'aho, h(x^) = {h(dynamics(x,u), 0.4)}'

x = [-0.1048257  -0.13827199], u = 8.90052782482294, x^ = [-0.04890911  1.16013997]
x = [ 0.47938163 -0.85620302], u = -6.056618009557773, x^ = [ 0.40844299 -1.46681006]


AssertionError: aho, h(x^) = -0.027438382519306526